# Build Processor

In [1]:
import torch
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# Load processed datasets

In [4]:
from datasets import load_from_disk

test_sampled_pro = load_from_disk("/media/chris/TheFlash/Master/data/test_sampled_pro")
train_sampled_pro = load_from_disk("/media/chris/TheFlash/Master/data/train_sampled_pro")

FileNotFoundError: Directory /media/chris/TheFlash/Master/data/test_sampled_pro not found

In [2]:
import transformers

# Get logging
transformers.logging.set_verbosity(10)

In [3]:
import torch
import numpy as np

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )
        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [6]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [7]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [8]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [9]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.05,
    mask_time_prob=0.05,
    layerdrop=0.05,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

loading configuration file https://huggingface.co/facebook/wav2vec2-large-xlsr-53/resolve/main/config.json from cache at /home/chris/.cache/huggingface/transformers/8508c73cd595eb416a1d517b90762416c0bc6cfbef529578079aeae4d8c14336.9c1655e075d9ef07cda724db675f9067777f6eb2dd14269a834fcde8a48e825a
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2Model"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.05,
  "final_dropout": 0.0,
  "g

In [10]:
#model.to('cuda:0')
model.freeze_feature_extractor()

In [11]:
#import torch
torch.cuda.is_available()

True

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./wav2vec2-large-xlsr-ger-chris",
  group_by_length=False,
  per_device_train_batch_size=5,
  gradient_accumulation_steps=3,
  evaluation_strategy="steps",
  fp16=True,
  num_train_epochs=30,
  save_steps=3000,
  eval_steps=3000,
  logging_steps=1000,
  learning_rate=1e-4,
  warmup_steps=500,
  save_total_limit=4,
  dataloader_num_workers=16,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [13]:
from transformers import Trainer

#model= model.to('cuda:0')
tokenizer=processor.feature_extractor

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_sampled_pro,
    eval_dataset=test_sampled_pro,
    tokenizer=tokenizer,
)

Using amp fp16 backend


In [14]:
trainer.train("./wav2vec2-large-xlsr-ger-chris/checkpoint-21000")

Loading model from ./wav2vec2-large-xlsr-ger-chris/checkpoint-21000).
***** Running training *****
  Num examples = 246525
  Num Epochs = 30
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 15
  Gradient Accumulation steps = 3
  Total optimization steps = 493050
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 1
  Continuing training from global step 21000
  Will skip the first 1 epochs then the first 13695 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


Step,Training Loss,Validation Loss,Wer
24000,0.142500,0.162015,0.266798
27000,0.137700,0.167592,0.266293
30000,0.135900,0.163439,0.265782
33000,0.132300,0.156836,0.258788
36000,0.124500,0.153671,0.257495
39000,0.121500,0.156014,0.259371
42000,0.119900,0.149734,0.254495
45000,0.121700,0.148483,0.255497
48000,0.119500,0.148480,0.254630
51000,0.108300,0.152217,0.253571


***** Running Evaluation *****
  Num examples = 15588
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ger-chris/checkpoint-24000
Configuration saved in ./wav2vec2-large-xlsr-ger-chris/checkpoint-24000/config.json
Model weights saved in ./wav2vec2-large-xlsr-ger-chris/checkpoint-24000/pytorch_model.bin
Configuration saved in ./wav2vec2-large-xlsr-ger-chris/checkpoint-24000/preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ger-chris/checkpoint-12000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 15588
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ger-chris/checkpoint-27000
Configuration saved in ./wav2vec2-large-xlsr-ger-chris/checkpoint-27000/config.json
Model weights saved in ./wav2vec2-large-xlsr-ger-chris/checkpoint-27000/pytorch_model.bin
Configuration saved in ./wav2vec2-large-xlsr-ger-chris/checkpoint-27000/preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ger-chris/che

KeyboardInterrupt: 